In [1]:
import json
import requests

import pandas as pd
import io

import datetime

import os

if not os.path.exists('../cache/bar-charts/'):
    os.makedirs('../cache/bar-charts/')
if not os.path.exists('../cache/bar-charts/people-data/'):
    os.makedirs('../cache/bar-charts/people-data/')
if not os.path.exists('../cache/bar-charts/people-geojson/'):
    os.makedirs('../cache/bar-charts/people-geojson/')
end_date = datetime.datetime.now()
last_monday = end_date - datetime.timedelta(days=end_date.weekday())
start_date = last_monday - datetime.timedelta(21)

dt_format = "%Y%m%d%H%M%S"
url = 'http://uoweb3.ncl.ac.uk:80/metadata/api/sensors?theme=People'


api_url = 'http://uoweb3.ncl.ac.uk/api/v1.1/sensors/{sensor_name}/data/csv/'
r = requests.get(url)

for item in r.json()['items']:
    json.dump(
        item['current_location']['geojson'],
        open(
            os.path.join('../cache/bar-charts/people-geojson/{sensor_name}.json'.format(sensor_name=item['name'])),
            'w'
        )
              )
    params = {
        'starttime': (start_date - datetime.timedelta(days=0)).strftime(dt_format),
        'endtime': (end_date - datetime.timedelta(days=0)).strftime(dt_format),
        'data_variable': ','.join(
            [variable['name'] for variable in item['variables'] if 'walk' in variable['name'].lower()]),
    }
    r = requests.get(api_url.format(sensor_name=item['name']), params)
    frame = pd.read_csv(io.StringIO(r.content.decode('utf-8')))

    frame.to_pickle('../cache/bar-charts/people-data/{sensor_name}.tar.gz'.format(sensor_name=item['name']))